<summary>
    <font size="2" color="orange"><b>1.1 Limpieza y Pre- procesamiento. </b></font>
</summary>

Grupo 01.

Para el grupo 1, se ajusta el formato de columnas manteniendo el resto de las variables y agregando como indice una columna con formato datetime, al final de esta libreta, se exporta el nuevo dataframe para

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os



<summary>
    <font size="4" color="orange"><b>1.2 Loading CENACE database: 49 input variables </b></font>
</summary>

<font size="3" color="palevioletred"><b>Exogenous Calendar Features </b></font>

* **FECHA** (yy-mm-dd): Date

"Holiday" (0|1) indicator:

* **Lunes_Festivo**: Holiday Monday

* **Martes_PostFestivo**: Day after holiday Monday

* **Semana_Santa**: Holy Week

* **1_Mayo**: May 1

* **10_Mayo**: May 10

* **16_Sep**: September 16

* **2_Nov.**: November 2

* **Pre-Navidad_y_new_year**: Day before Christmas or New Year

* **Navidad_y_new_year**: Christmas or New Year

* **Post-Navidad_y_new_year**: Day after Christmas or New Year

<font size="3" color="palevioletred"><b>Endogenous Feature</b></font>

* **DEM_GCRNO_H$i$** (MW): Load energy demand in GCRNO (Gerencia de Control de Noroeste)  zone from hour $i$ to hour $i+1$ of the corresponding date, for $i=0,\dots 23$.


<font size="3" color="palevioletred"><b>Exogenous Meteorological Features</b></font>

* **Tmax-Cab**, **Tmin-Cab**, **Tmax-HMO**, **Tmin-HMO**, **Tmax-OBR**, **Tmin-OBR**,**Tmax-LMO**, **Tmin-LMO**, **Tmax-CUL**, **Tmin-CUL** ($^\circ$C): Maximum and Minimum Temperature in Caborca, Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.

* **PREC_HMO_mm**, **PREC_OBR_mm**, **PREC_LMO_mm**, **PREC_CUL_mm**  (mm/h): Precipitation in Hermosillo, Ciudad Obregón, Los Mochis and Culiacán, respectively.



In [3]:
# Importing load energy consumption CENACE database
gcrno = pd.read_csv('../Grupo_01/data1/DATASETGCRNODIARIO20230118.csv', delimiter=",")
gcrno.columns

Index(['FECHA', 'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9',
       'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19',
       'H20', 'H21', 'H22', 'H23', 'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR',
       'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB', 'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO',
       'TMIN-CUL', 'PREC_HMO_MM', 'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM',
       'LUNES_FESTIVO', 'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO',
       '10_MAYO', '16_SEP', '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR',
       'NAVIDAD_Y_NEW_YEAR', 'POST-NAVIDAD_Y_NEW_YEAR'],
      dtype='object')

In [4]:
#set working directory  
os.chdir('./data1')
print(os.getcwd())

/Users/yhmve/energy_demand_anlys/Grupo_01/data1


<summary>
    <font size="4" color="orange"><b>1.3. Dataframe rearrangement</b></font>
</summary>

The  hours part of dataframe will be transform in a new one with:

* *INSTANCES* (index):
 **FECHA-HORA** (Date-Hour) specified in the format yyyy-mm-dd hh:00:00
    
    
* *FEATURES*: 

    **DEMAND** Load energy demand
    
    **DIA** (Day)
       0 Monday 
       1 Tuesday 
       2 Wednesday 
       3 Thursday 
       4 Friday 
       5 Saturday 
       6 Sunday
    
    **HORA** (Hour 0–23)
      
    **MES** (Month)
       1 January
       2 February
       3 March
       4 April
       5 May
       6 Jun
       7 July
       8 August
       9 September
       10 Octuber
       11 November
       12 December
    
    And the following characteristics with constant value with respect to the day **Tmax-Cab**, **Tmin-Cab**, **Tmax-HMO**, **Tmin-HMO**, **Tmax-OBR**, **Tmin-OBR**,**Tmax-LMO**, **Tmin-LMO**, **Tmax-CUL**, **Tmin-CUL**, **PREC_HMO_MM**, **PREC_OBR_MM**, **PREC_LMO_MM**, **PREC_CUL_MM**, 

In [5]:
# Transposing hours columns from the original dataframe into rows

consumption = gcrno.melt(
    id_vars= ['FECHA'],
    value_vars= [f'H{i}' for i in range(24)],
    var_name="HORA",
    value_name="DEMANDA"
).replace(
    {f'H{i}': i for i in range(24)}
    )

In [6]:
# Creating Day, Hour and Month columns
consumption['FECHA']= pd.to_datetime(consumption['FECHA'], format='%d/%m/%Y')
consumption.index = consumption.FECHA + pd.to_timedelta(consumption.HORA, unit='h')
consumption.sort_index(inplace=True)
consumption.drop(columns=['HORA'], inplace=True)
consumption = consumption.asfreq('h', method='pad')
consumption['Date_time'] = consumption.index
consumption["Day"] = consumption.index.weekday
consumption["Hour"] = consumption.index.hour
consumption["Month"] = consumption.index.month
consumption["Year"] = consumption.index.year


In [7]:
#updating column titles in consumo
consumption.rename(columns={"FECHA":"Date",'DEMANDA':'Energy_Demand'}, inplace = True)

In [8]:
# Adding columns of exogenous variables
exogenous = gcrno[['FECHA', 'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR',
       'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB', 'TMIN-HMO', 'TMIN-OBR', 
       'TMIN-LMO','TMIN-CUL',  'PREC_HMO_MM','PREC_OBR_MM', 
       'PREC_LMO_MM', 'PREC_CUL_MM', 'LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR']]

In [9]:
#updating column titles in exogenous
exogenous.columns = [col.title() for col in exogenous.columns]
exogenous.head(2)

,Fecha,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,...,Lunes_Festivo,Martes_Postfestivo,Semana_Santa,1_Mayo,10_Mayo,16_Sep,2_Nov.,Pre-Navidad_Y_New_Year,Navidad_Y_New_Year,Post-Navidad_Y_New_Year
0,01/01/2007,21.0,22.0,25.0,30.0,29.0,2.0,9.0,8.0,10.0,...,0,0,0,0,0,0,0,0,1,0
1,02/01/2007,21.0,22.0,22.0,22.0,27.0,2.0,7.0,7.0,11.0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
exogenous.rename(columns={"Fecha":"Date",'Lunes_Festivo':'Monday_Holiday',
       'Martes_Postfestivo':'Tuesday_Aft_Hol', 'Semana_Santa':'Easter_week', '1_Mayo':'May_1s', 
       '10_Mayo':'May_10t', '16_Sep':'Sept_16','2_Nov.':'Nov_2nd',
       'Pre-Navidad_Y_New_Year':'Before_Christmas_NY', 'Navidad_Y_New_Year':'Christmas_NY',
       'Post-Navidad_Y_New_Year':'After_Christmas_NY'}, inplace = True)

In [11]:
consumption.columns

Index(['Date', 'Energy_Demand', 'Date_time', 'Day', 'Hour', 'Month', 'Year'], dtype='object')

In [12]:
exogenous['Date']= pd.to_datetime(exogenous['Date'], format='%d/%m/%Y')
consumption_com = pd.merge(consumption, exogenous, on='Date', how='left')

In [13]:
consumption_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140688 entries, 0 to 140687
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date                 140688 non-null  datetime64[ns]
 1   Energy_Demand        140688 non-null  float64       
 2   Date_time            140688 non-null  datetime64[ns]
 3   Day                  140688 non-null  int64         
 4   Hour                 140688 non-null  int64         
 5   Month                140688 non-null  int64         
 6   Year                 140688 non-null  int64         
 7   Tmax-Cab             140688 non-null  float64       
 8   Tmax-Hmo             140688 non-null  float64       
 9   Tmax-Obr             140688 non-null  float64       
 10  Tmax-Lmo             140688 non-null  float64       
 11  Tmax-Cul             140688 non-null  float64       
 12  Tmin-Cab             140688 non-null  float64       
 13  Tmin-Hmo      

In [14]:
# Setting as index the Date_time
consumption_com.set_index("Date_time", inplace=True)
consumption_com=consumption_com.asfreq('h')

In [15]:
consumption_com.tail(3)

,Date,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2023-01-18 21:00:00,2023-01-18,2513.06,2,21,1,2023,17.6,21.5,22.0,24.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-18 22:00:00,2023-01-18,2450.30,2,22,1,2023,17.6,21.5,22.0,24.0,...,0,0,0,0,0,0,0,0,0,0
2023-01-18 23:00:00,2023-01-18,2332.95,2,23,1,2023,17.6,21.5,22.0,24.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
consumption_com['Day'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [17]:
# Verifying existence of missing data 
consumption_com.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 140688 entries, 2007-01-01 00:00:00 to 2023-01-18 23:00:00
Freq: H
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date                 140688 non-null  datetime64[ns]
 1   Energy_Demand        140688 non-null  float64       
 2   Day                  140688 non-null  int64         
 3   Hour                 140688 non-null  int64         
 4   Month                140688 non-null  int64         
 5   Year                 140688 non-null  int64         
 6   Tmax-Cab             140688 non-null  float64       
 7   Tmax-Hmo             140688 non-null  float64       
 8   Tmax-Obr             140688 non-null  float64       
 9   Tmax-Lmo             140688 non-null  float64       
 10  Tmax-Cul             140688 non-null  float64       
 11  Tmin-Cab             140688 non-null  float64       
 12  Tmin-Hmo             140688 no

<summary>
    <font size="4" color="orange"><b>2. Exploring variables</b></font>
</summary>

<br/>
<summary>

    <font size="3" color="palevioletred"><b>Energy Demand</b></font>
</summary>

In [18]:
#looking for more information on endogenous Features Demanda
consumption_com['Energy_Demand'].describe()

count    140688.000000
mean       2499.355915
std         818.947994
min         959.000000
25%        1869.000000
50%        2337.455000
75%        3039.000000
max        5402.720000
Name: Energy_Demand, dtype: float64

<summary>
    <font size="3" color="palevioletred"><b>Exogenous Meteorological Features</b></font>
</summary>

In [19]:
consumption_com.head(2)

,Date,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,2007-01-01,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,...,0,0,0,0,0,0,0,0,1,0
2007-01-01 01:00:00,2007-01-01,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
#looking for more information on enxogenous Features Tmax & Tmin
consumption_com[['Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul']].describe()

,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,Tmin-Cul
count,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000,140688.000000
mean,32.862801,33.514630,34.434591,32.342767,33.896165,16.371192,18.420218,18.536718,18.941286,20.235003
std,7.854035,6.646942,5.966135,4.882281,4.088913,7.954089,7.083570,6.798394,6.127298,5.477858
min,9.000000,8.000000,12.000000,12.000000,17.000000,-7.000000,-3.000000,2.000000,1.000000,1.000000
25%,27.000000,28.500000,30.000000,29.000000,31.000000,10.000000,13.000000,13.000000,13.850000,16.000000
50%,33.540000,34.050000,35.000000,33.000000,34.095000,16.000000,18.000000,18.000000,18.000000,20.470000
75%,39.500000,39.000000,39.000000,36.000000,37.000000,23.400000,25.000000,25.000000,25.000000,25.000000
max,50.000000,49.100000,47.000000,45.000000,44.000000,33.000000,34.000000,42.500000,37.000000,32.000000


In [22]:
#To export file remote  "date"
consumption_com.drop('Date', inplace=True, axis=1)
consumption_com.head(2)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,0,1,0
2007-01-01 01:00:00,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
#Export file
#send combined dataframe to file
consumption_com.to_csv (r'group01_all.csv', index = True)